In [2]:
import sys 
sys.path.append('/Users/justinvhuang/Desktop/CSE-6242-Group-Project')

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores import Chroma
from utils.textpreprocessing import TextPreprocessor

In [4]:
textprepo = TextPreprocessor()

In [5]:
encode_kwargs = {"normalize_embeddings": True}
embedding_function = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={"device": "cpu"},
    encode_kwargs=encode_kwargs,
)

In [6]:
db_faiss = FAISS.load_local("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/vector_database_creation/faiss_anime_index_v3", embeddings = embedding_function)

In [7]:
import yaml

# Load API key from config.yaml
with open("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/app/config.yaml", "r") as file:
    config = yaml.safe_load(file)

api_key = config["api_key"]

In [102]:
import google.generativeai as genai
userdata = {"GOOGLE_API_KEY": api_key}
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [103]:
#query_token = textprepo.preprocess_text(query)

def filter_tokens(metadata):
        metadata_tokens = metadata.get("tokens", [])
        return any(token in metadata_tokens for token in query_token) or metadata["score"] > 5.0


In [104]:
retriever = db_faiss.as_retriever(search_kwargs={"k": 50, "filter": filter_tokens})

In [105]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI


In [106]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0,google_api_key = GOOGLE_API_KEY)

In [107]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [117]:
from langchain_core.prompts import PromptTemplate

template = """You are an anime afficinado who went deep weeb under your mothers basement who knows all about anime and japanese culture. 
Use the following pieces of context to summarize different animes at the end.
Use only use stuff from the context to answer the answer and summarize the question. 
Recommend up to 5 anime and provide a one sentence description for each anime.
Always say "Thanks for using the anime recommender from Casual Correlations!" at the end of the answer.

Below is an example
```
Question: can you recommend me 5 animes from the 90

Summary: 

1. Neon Genesis Evangelion: A group of teenagers pilot giant robots to protect humanity from mysterious creatures known as Angels, in a series renowned for its complex characters and psychological themes.

2. Cowboy Bebop: This space-western series follows the adventures of a group of bounty hunters aboard the spaceship Bebop, with a jazzy soundtrack and stylish animation.

3. Dragon Ball Z: Goku and his friends defend Earth from powerful foes in this action-packed series that became a major phenomenon.

4. Sailor Moon: Usagi Tsukino transforms into the magical warrior Sailor Moon to battle evil forces and protect the world, known for its empowering themes and strong female characters.

5. Ghost in the Shell: Major Motoko Kusanagi and her team investigate cybercrimes in a future where humanity is interconnected with technology, exploring questions of identity and consciousness.

Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes from studio ghibli

1. My Neighbor Totoro (1988): Directed by Hayao Miyazaki, this heartwarming tale follows two young sisters who encounter friendly forest spirits in rural Japan.

2. Spirited Away (2001): Also directed by Hayao Miyazaki, Spirited Away tells the story of a young girl named Chihiro who becomes trapped in a mysterious and magical world, where she must work in a bathhouse for spirits to rescue her parents.

3. Princess Mononoke (1997): Directed by Hayao Miyazaki, this epic fantasy film explores the conflict between industrialization and nature, as a young prince becomes entangled in a struggle between forest gods and human settlers.

4. Howl's Moving Castle (2004): Directed by Hayao Miyazaki, this enchanting film follows a young woman named Sophie who is cursed by a witch and seeks refuge in the moving castle of the wizard Howl.

5. Kiki's Delivery Service (1989): Directed by Hayao Miyazaki, this charming film follows a young witch named Kiki who moves to a new town and starts a delivery service using her flying broomstick.

Thanks for using the anime recommender from Casual Correlations!

Question: what are some animes similar to one piece

1. Fairy Tail: This series follows the adventures of Natsu Dragneel and his friends in the Fairy Tail guild as they take on various quests and battles in the magical land of Fiore. Like One Piece, it features a diverse cast of characters, epic battles, and a strong sense of camaraderie.

2. Naruto: Naruto follows the journey of Naruto Uzumaki, a young ninja with dreams of becoming the strongest ninja and leader of his village, the Hokage. It features a similar blend of action, humor, and heartfelt moments, as well as a focus on friendship and determination.

3. Hunter x Hunter: This series follows Gon Freecss, a young boy who aspires to become a Hunter like his father, as he embarks on various adventures and challenges in search of his father and his own identity. It shares themes of friendship, adventure, and personal growth with One Piece.

4. Bleach: Bleach follows Ichigo Kurosaki, a teenager with the ability to see ghosts, as he becomes a Soul Reaper and battles evil spirits known as Hollows. Like One Piece, it features intense battles, supernatural elements, and a large cast of characters.

5. One Punch Man: While it's more of a parody of traditional shonen anime, One Punch Man shares some similarities with One Piece in its action-packed battles and larger-than-life characters. It follows Saitama, a hero who can defeat any opponent with a single punch, as he seeks a worthy opponent and navigates the world of superheroes.

Thanks for using the anime recommender from Casual Correlations!
```

{context}

Question: {question}

Summary:"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [118]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [120]:
query = 'what are some anime like totoro'
query_token = textprepo.preprocess_text(query)
print(rag_chain.invoke(query))

1. My Neighbor Totoro (1988): Directed by Hayao Miyazaki, this heartwarming tale follows two young sisters who encounter friendly forest spirits in rural Japan.

2. Spirited Away (2001): Also directed by Hayao Miyazaki, Spirited Away tells the story of a young girl named Chihiro who becomes trapped in a mysterious and magical world, where she must work in a bathhouse for spirits to rescue her parents.

3. Princess Mononoke (1997): Directed by Hayao Miyazaki, this epic fantasy film explores the conflict between industrialization and nature, as a young prince becomes entangled in a struggle between forest gods and human settlers.

4. Howl's Moving Castle (2004): Directed by Hayao Miyazaki, this enchanting film follows a young woman named Sophie who is cursed by a witch and seeks refuge in the moving castle of the wizard Howl.

5. Kiki's Delivery Service (1989): Directed by Hayao Miyazaki, this charming film follows a young witch named Kiki who moves to a new town and starts a delivery se